In [1]:
# New notebook for BERT until it works properly.

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_pickle("./data/cleaned_submissions.pkl")
data = data.loc[data["sentiment"] != "", ["text", "sentiment"]] # Only get labeled instances

# Encode labels
lenc = LabelEncoder()
y = lenc.fit_transform(data["sentiment"])

# Vectorize text using tfidf
tfidf = TfidfVectorizer(preprocessor=' '.join, lowercase=False, min_df=5) # min_df = Minimum occurance of words
X = tfidf.fit_transform(data["text"])

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [5]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
model.summary()

Downloading: 100%|██████████| 570/570 [00:00<00:00, 83.5kB/s]
Downloading: 100%|██████████| 511M/511M [00:17<00:00, 31.0MB/s]
2021-11-18 17:03:25.596225: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [16]:
import tensorflow as tf

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(X_train, epochs = 1)

2021-11-18 17:23:25.002921: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at serialize_sparse_op.cc:384 : INVALID_ARGUMENT: indices[1] = [0,2642] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.




InvalidArgumentError: indices[1] = [0,2642] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]

In [11]:
print(type(X_train))

<class 'scipy.sparse.csr.csr_matrix'>
